In [1]:
import pandas as pd
import numpy as np

In [2]:
use_data = pd.read_csv('../data_merge/merged_data.csv')

In [3]:
use_data = use_data.drop(["출발시간", "timestamp", "구분"], axis=1)

In [4]:
use_data = use_data.reindex(['날짜', '항공사', '편명', '계획시간', 'wind_direction', 'wind_speed',
       'temperature', 'pressure', '현황'], axis=1)

In [5]:
use_data.nunique()

날짜                 366
항공사                 87
편명                1114
계획시간               280
wind_direction      38
wind_speed          28
temperature         49
pressure           137
현황                   4
dtype: int64

---

In [6]:
    # 1. 날짜 처리
use_data['날짜'] = pd.to_datetime(use_data['날짜'], format='%Y%m%d')
use_data['요일'] = use_data['날짜'].dt.dayofweek  # 0: 월요일, 6: 일요일
use_data['계절'] = use_data['날짜'].dt.month % 12 // 3  # 0: 겨울, 1: 봄, 2: 여름, 3: 가을

In [7]:
# 2. 시간대 분류
use_data['시간'] = pd.to_datetime(use_data['계획시간'], format='%H:%M').dt.hour
use_data['시간대'] = pd.cut(use_data['시간'], bins=[0, 6, 12, 18, 24], labels=[0, 1, 2, 3], right=False)

In [9]:
# 항공사 및 편명 매핑 테이블 생성
airline_mapping = {name: code for code, name in enumerate(use_data['항공사'].astype('category').cat.categories)}
flight_mapping = {name: code for code, name in enumerate(use_data['편명'].astype('category').cat.categories)}
use_data['항공사'] = use_data['항공사'].astype('category').cat.codes
use_data['편명'] = use_data['편명'].astype('category').cat.codes

In [11]:
# 4. 목표 변수 변환
status_mapping = {"출발": 0, "지연": 1, "취소": 2, "회항": 3}
use_data['현황'] = use_data['현황'].map(status_mapping)

In [12]:
# 날짜를 숫자로 변환
use_data['날짜'] = pd.to_datetime(use_data['날짜'])
use_data['연도'] = use_data['날짜'].dt.year
use_data['월'] = use_data['날짜'].dt.month
use_data['일'] = use_data['날짜'].dt.day
use_data.drop(columns=['날짜'], inplace=True)  # 원본 날짜 컬럼 삭제

In [13]:
# 계획시간을 분 단위로 변환
use_data['계획시간'] = pd.to_datetime(use_data['계획시간'], format='%H:%M')
use_data['시간(분)'] = use_data['계획시간'].dt.hour * 60 + use_data['계획시간'].dt.minute
use_data.drop(columns=['계획시간'], inplace=True)  # 원본 시간 컬럼 삭제

In [14]:
use_data = use_data.reindex(['항공사', '편명', 'wind_direction', 'wind_speed', 'temperature', 'pressure', '요일', '계절', '시간', '시간대', '연도', '월', '일', '시간(분)',
       '현황'], axis=1)

In [15]:
use_data.to_csv("./processed_merged_data.csv", index=False)